In [1]:
from ucimlrepo import fetch_ucirepo
import pandas as pd
import numpy as np

In [2]:
# Pull in remote dataset from UCI ML repo
raw_data = fetch_ucirepo(id=849) 

In [3]:
# Create flat table
raw_df: pd.DataFrame = raw_data.data.features.merge(raw_data.data.targets,
                                                    how='outer',
                                                    left_index=True,
                                                    right_index=True)

# Relabeling with friendly column headers
new_colnames = ["datetime",
                "temperature",
                "humidity",
                "wind_speed",
                "general_diffuse_flows",
                "diffuse_flows",
                "zone_1_power",
                "zone_2_power",
                "zone_3_power"]

colname_updates = {orig_colnames: new_colnames for orig_colnames,
                   new_colnames in zip(raw_df.columns, new_colnames)}

tetoun_power: pd.DataFrame = raw_df.rename(columns=colname_updates)

# Parsing new date/time features
tetoun_power["datetime"] = pd.to_datetime(tetoun_power["datetime"])

tetoun_power = tetoun_power.assign(
    year=tetoun_power["datetime"].dt.year,
    month=tetoun_power["datetime"].dt.month,
    day=tetoun_power["datetime"].dt.day,
    hour=tetoun_power["datetime"].dt.hour,
    minute=tetoun_power["datetime"].dt.minute,
    day_of_week=tetoun_power["datetime"].dt.dayofweek,
    day_of_year=tetoun_power["datetime"].dt.dayofyear,
    weekofyear=tetoun_power["datetime"].dt.isocalendar().week
)

# Calculating new features
tetoun_power["total_power"] = tetoun_power["zone_1_power"] + \
    tetoun_power["zone_2_power"] + tetoun_power["zone_3_power"]

In [5]:
tetoun_power.head(2)

,datetime,temperature,humidity,wind_speed,general_diffuse_flows,diffuse_flows,zone_1_power,zone_2_power,zone_3_power,year,month,day,hour,minute,day_of_week,day_of_year,weekofyear,total_power
0,2017-01-01 00:00:00,6.559,73.8,0.083,0.051,0.119,34055.69620,16128.87538,20240.96386,2017,1,1,0,0,6,1,52,70425.53544
1,2017-01-01 00:10:00,6.414,74.5,0.083,0.070,0.085,29814.68354,19375.07599,20131.08434,2017,1,1,0,10,6,1,52,69320.84387
